## PSYCH 186B Final Project
### Predicting Recidiism with Multi Layer Perceptrons
Winter 2023 Bailey Swain

### What is the problem?
The United States has the highest total number of incarcerated people around the world. Recidivism is the tendency of a convicted criminal to reoffend. 44% of formerly incarcerated people return to prison within the same year of being released. As time progresses the statistics just get worse with 68% returning within 3 years, 79% after 6 years, and 83% after 9. With recidivism rates as alarming as these many cities across America grapple with how to ethically integrate those who have been released back into society without compromising public safety.

### Why is it interesting?
This is interesting because the United States Federal Justice System has been seeking a way to empirically assess the risk of recidivism since the early twentieth century. [1][2] At the time the process used a correlation between age and criminal history to predict recidivism. By the 1970s these were extremely popular. Today the tools used to assess risk are able to use a large set of data including things like substance abuse, and employment. 

These tools are incredibly powerful because they are used to make very high stake decisions like whether someone will be granted bail, confinement form, community supervision conditions and revocation, being released between the time of arrest and trial. Machine learning is already widely used in the criminal justice system by using historical data to find patterns and form predictive accuracy. Something that makes it so interesting to me is the possibilities when it comes to discorverying new ways to make this type of algorithm impartial.


### What has been done in literature?
In the last 5 years there has been great efforts to advance recidivism prediction using machine learning. Northpointe’s tool, called COMPAS (Correctional Offender Management Profiling for Alternative Sanctions) has gained a lot of attention because it is a widely used commercial tool. However, there have been many data scientists who say the algorithm is racially biased against black offenders. 

Sarah Desmarais and Jay Singh studied almost 20 varying different recidivism risk tools in 2013. They found that most of the time these Machine Learning Algorithims being used had never been tested for validity. 

Since then the largest exploration of racial bias in the US recidivism Machine Learning algorithms was done at the University of California, Berkeley. In 2016 they reviewed 34,000 federal offenders and tested the predictive validity of a tool that was used by the federal courts to aid in probation and parole officers.

### Library Import

In [39]:
#arrays 
import pandas as pd
import numpy as np

# plots
import seaborn as sns
import matplotlib.pyplot as plt

# ML
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression,LogisticRegression,Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import mutual_info_classif
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.models import Sequential
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, GridSearchCV,train_test_split,cross_val_score,cross_validate
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, roc_auc_score,roc_curve, auc, brier_score_loss
from sklearn import tree
import xgboost as xgb
from xgboost import XGBClassifier
from itertools import cycle
from sklearn.metrics import confusion_matrix
import pydotplus
import graphviz

import keras
import tensorflow as tf
import tensorflow_probability as tfp

from keras.layers import Dense, Input, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation, Embedding
from tensorflow.keras import backend as K

#NN
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

My goal is to predict if a formerly incarcerated person will go bck to prison within 3 years based on their data. Knowing this is important because it will allow us to identify high risk individuals such that we can allocate more resouces to them or take time to focus on their education/habits. In order to evaluate if the model used is worth the resources, we can look at how well it stacks up against other prediction models and find ways to best tune it assuming we already want to find a way to identify high risk individuals.

In [40]:
from sklearn.metrics import roc_auc_score
def y_roc(estimator,X):
    y_scores=[]
    for list in estimator.predict(X):
        y_scores.append(list[1])
    return y_scores
def y_roc_regression(estimator,X):
    y_scores=[]
    for list in estimator.predict(X):
        y_scores.append(list)
    return y_scores
def cv_roc_plot(estimator,X,y):
    cv = StratifiedKFold(n_splits=4,shuffle=False)
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0,1,100)
    for train,test in cv.split(X,y):
        prediction = estimator.fit(X.iloc[train],y.iloc[train]).predict(X.iloc[test])
        fpr, tpr, t = roc_curve(y.iloc[test], prediction[:, 1])
        tpr[0]=0
        tpr[-1]=1
        tprs.append(interp(mean_fpr, fpr, tpr))
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
    mean_tpr = np.mean(tprs, axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    return mean_fpr, mean_tpr, mean_auc
def brier_score(y_prob_raw,y_true):
    y_prob=[prob[1] for prob in y_prob_raw]
    if len(y_prob)!=len(y_true):
        print('Error: two lists must have same length')
        return
    out = 0
    for prob_1,y in zip(y_prob,y_true):
        out+=(prob_1-y)**2
    return out/len(y_prob)
def get_prob_1(y_prob_raw):
    return [prob[1] for prob in y_prob_raw]
def aver_prob(prob_lists):
    n=len(prob_lists)
    return np.sum(np.array(prob_lists),0)/n

### Data Import

In [41]:
# Read in the dataset

url = 'https://raw.githubusercontent.com/bswain7/MLP_Recidivism/nij_data/NIJ_s_Recidivism_Challenge_Full_Dataset.csv'
recidivism = pd.read_csv(url)

### Initial Data Processing
##### Changing and data to integers

In [42]:
#Drop unnecessary columns
cols_2_drop =['Violations_ElectronicMonitoring',
       'Violations_Instruction', 'Violations_FailToReport',
       'Violations_MoveWithoutPermission',
       'Avg_Days_per_DrugTest', 'DrugTests_THC_Positive',
       'DrugTests_Cocaine_Positive', 'DrugTests_Meth_Positive',
       'DrugTests_Other_Positive', 'Percent_Days_Employed', 'Jobs_Per_Year',
       'Employment_Exempt']

recidivism = recidivism.drop(cols_2_drop, axis=1)
recidivism.columns

Index(['ID', 'Gender', 'Race', 'Age_at_Release', 'Residence_PUMA',
       'Gang_Affiliated', 'Supervision_Risk_Score_First',
       'Supervision_Level_First', 'Education_Level', 'Dependents',
       'Prison_Offense', 'Prison_Years', 'Prior_Arrest_Episodes_Felony',
       'Prior_Arrest_Episodes_Misd', 'Prior_Arrest_Episodes_Violent',
       'Prior_Arrest_Episodes_Property', 'Prior_Arrest_Episodes_Drug',
       'Prior_Arrest_Episodes_PPViolationCharges',
       'Prior_Arrest_Episodes_DVCharges', 'Prior_Arrest_Episodes_GunCharges',
       'Prior_Conviction_Episodes_Felony', 'Prior_Conviction_Episodes_Misd',
       'Prior_Conviction_Episodes_Viol', 'Prior_Conviction_Episodes_Prop',
       'Prior_Conviction_Episodes_Drug',
       'Prior_Conviction_Episodes_PPViolationCharges',
       'Prior_Conviction_Episodes_DomesticViolenceCharges',
       'Prior_Conviction_Episodes_GunCharges', 'Prior_Revocations_Parole',
       'Prior_Revocations_Probation', 'Condition_MH_SA', 'Condition_Cog_Ed',
     

In [43]:
# Change the dtype of the feature from object to integer
recidivism['Residence_PUMA']=recidivism['Residence_PUMA'].astype('category')
recidivism['Residence_PUMA']=recidivism['Residence_PUMA'].cat.codes
recidivism['Age_at_Release']=recidivism['Age_at_Release'].apply(lambda x: int(x[:2]))
recidivism['Dependents']=recidivism['Dependents'].apply(lambda x: int(x[:1]))
recidivism['Prior_Arrest_Episodes_Felony']=recidivism['Prior_Arrest_Episodes_Felony'].apply(lambda x: int(x[:2]))
recidivism['Prior_Arrest_Episodes_Drug']=recidivism['Prior_Arrest_Episodes_Drug'].apply(lambda x: int(x[:2]))
recidivism['Prior_Arrest_Episodes_Misd']=recidivism['Prior_Arrest_Episodes_Misd'].apply(lambda x: int(x[:2]))
recidivism['Prior_Arrest_Episodes_Violent']=recidivism['Prior_Arrest_Episodes_Violent'].apply(lambda x: int(x[:2]))
recidivism['Prior_Arrest_Episodes_Property']=recidivism['Prior_Arrest_Episodes_Property'].apply(lambda x: int(x[:2]))
recidivism['Prior_Arrest_Episodes_PPViolationCharges']=recidivism['Prior_Arrest_Episodes_PPViolationCharges'].apply(lambda x: int(x[:2]))
recidivism['Prior_Conviction_Episodes_Felony']=recidivism['Prior_Conviction_Episodes_Felony'].apply(lambda x: int(x[:1]))
recidivism['Prior_Conviction_Episodes_Misd']=recidivism['Prior_Conviction_Episodes_Misd'].apply(lambda x: int(x[:1]))
recidivism['Prior_Conviction_Episodes_Prop']=recidivism['Prior_Conviction_Episodes_Prop'].apply(lambda x: int(x[:1]))
recidivism['Prior_Conviction_Episodes_Drug']=recidivism['Prior_Conviction_Episodes_Drug'].apply(lambda x: int(x[:1]))
recidivism['Delinquency_Reports']=recidivism['Delinquency_Reports'].apply(lambda x: int(x[:1]))
recidivism['Program_Attendances']=recidivism['Program_Attendances'].apply(lambda x: int(x[:2]))
recidivism['Program_UnexcusedAbsences']=recidivism['Program_UnexcusedAbsences'].apply(lambda x: int(x[:1]))
recidivism['Residence_Changes']=recidivism['Residence_Changes'].apply(lambda x: int(x[:1]))

    # scale the columns which are not bool or category
      
scaler = StandardScaler()
scaling_set=[]
for column in recidivism.columns:
    if recidivism[column].dtype == object:
        recidivism[column]=recidivism[column].astype('category')
        recidivism[column]=recidivism[column].cat.codes
    elif recidivism[column].dtype in ['int64','float32','float64'] :
        scaling_set+=[column]
recidivism[scaling_set]=scaler.fit_transform(recidivism[scaling_set].values)

In [44]:
# remove outliers
recidivism=recidivism.drop(index=recidivism[recidivism.Supervision_Risk_Score_First.isnull()].index)
recidivism=recidivism.drop(index=set(recidivism[recidivism.Supervision_Risk_Score_First.isnull()].index) & 
                         set(recidivism[recidivism.Prison_Offense .isnull()].index))
recidivism=recidivism.reset_index(drop=True)
 
    
# impute missing value 'Supervision_Level_First' and 'Prison_Offense' with relative feature

for missing_column in ['Supervision_Level_First','Prison_Offense']:
    test_index=recidivism[recidivism[missing_column]==-1].index
    train_index=recidivism[recidivism[missing_column]!=-1].index
    X=recidivism.drop(columns=[missing_column])
    y=recidivism[missing_column]
    
    logreg = LogisticRegression()
    logreg.fit(X.iloc[train_index,:],y[train_index])
    recidivism.loc[test_index,missing_column]=logreg.predict(X.iloc[test_index,:])
    recidivism[missing_column]=recidivism[missing_column].astype('category')
    recidivism[missing_column]=recidivism[missing_column].cat.codes


/Users/puddinpop/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/puddinpop/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

##### This model can act as a baseline for recidivism.

In [45]:
# split features and recidivism


features = recidivism.drop(columns=["Recidivism_Within_3years"])
target = recidivism.Recidivism_Within_3years

In [46]:
# split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
  features, target, test_size=0.33, random_state=42)

# convert to tensor arrays
X_train_np = X_train.values
X_test_np = X_test.values
y_train_np = y_train.values
y_test_np = y_test.values

In [47]:
class Data(recidivism):
  def __init__(self, X_train, y_train):
    # need to convert float64 to float32 else 
    # will get the following error
    # RuntimeError: expected scalar type Double but found Float
    self.X = torch.from_numpy(X_train.astype(np.float32))
    # need to convert float64 to Long else 
    # will get the following error
    # RuntimeError: expected scalar type Long but found Float
    self.y = torch.from_numpy(y_train).type(torch.LongTensor)
    self.len = self.X.shape[0]
  
  def __getitem__(self, index):
    return self.X[index], self.y[index]
  def __len__(self):
    return self.len

In [48]:
batch_size = 50
traindata = Data(X_train_np,y_train_np)
trainloader = DataLoader(traindata, batch_size=batch_size, 
                         shuffle=True, num_workers=0) # split even smaller

In [49]:
# Network class parameters
input_dim = len(features.columns)
output_dim = 3
hidden = (input_dim + output_dim) // # bt input and output dims

In [50]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.leakyrelu = nn.LeakyReLU(0.001)
    self.linear1 = nn.Linear(input_dim,hidden)
    self.linear2 = nn.Linear(hidden,output_dim)

  def forward(self, x):
    x = self.leakyrelu(self.linear1(x))
    x = self.linear2(x)
    return x

In [51]:
clf = Network()
print(clf.parameters)

<bound method Module.parameters of Network(
  (leakyrelu): LeakyReLU(negative_slope=0.001)
  (linear1): Linear(in_features=41, out_features=22, bias=True)
  (linear2): Linear(in_features=22, out_features=3, bias=True)
)>


In [52]:
criterion = nn.CrossEntropyLoss() # best for categories
optimizer = torch.optim.SGD(clf.parameters(), lr=0.001, weight_decay=1e-5)

In [53]:
# confusion matrix for evaluation - will be used later on
def plot_confusion_matrix(matrix,title=""): 
    #put the heatmap into the figure 
    sns.heatmap(data=matrix, annot=True, cmap="spring")
    status=["0","1"]
    axis_ticks=np.arange(len(status))+0.4
    
    #sets x axis ticks to species names
    plt.xticks(axis_ticks,status) 
    
    #sets y axis ticks to species names 
    plt.yticks(axis_ticks,status) 
    plt.title(title)
    plt.ylabel("Actual Label")
    plt.xlabel("Predicted Label")

### Multi Layer Perceptron with Mean Sqaured Error

In [35]:
#(Baseline) Multi Layer Perceptron with Mean Squared Error

from sklearn.linear_model import LinearRegression,LogisticRegression


X_train1,X_test1,y_train1,y_test1=train_test_split(recidivism.drop(['ID','Recidivism_Within_3years','Recidivism_Arrest_Year1','Recidivism_Arrest_Year2','Recidivism_Arrest_Year3'],axis=1),recidivism['Recidivism_Arrest_Year1'],test_size=0.2)
fpr_list,tpr_list,auc_list=dict(),dict(),dict()

    
mse_perceptron = Sequential()
n_cols = X_train1.shape[1]
mse_perceptron.add(BatchNormalization(input_shape=(n_cols,)))
mse_perceptron.add(Dense(70, activation= 'linear'))
mse_perceptron.add(Dropout(0.3))
mse_perceptron.add(Dense(50, activation= 'relu'))
mse_perceptron.add(Dropout(0.3))
mse_perceptron.add(Dense(50, activation= 'relu'))
mse_perceptron.add(Dropout(0.3))

#model.add(Dense(100, activation='relu'))
#model.add(Dropout(0.3))
#model.add(Dense(100, activation='linear'))
mse_perceptron.add(BatchNormalization())
mse_perceptron.add(Dense(2, activation='softmax'))

early_stopping_monitor = EarlyStopping(patience=50)
sgd = keras.optimizers.SGD(lr=.001, decay=2e-4, momentum=0.9, nesterov=True)
mse_perceptron.compile(loss = 'mean_squared_error',optimizer='sgd', metrics=['accuracy'])
history=mse_perceptron.fit(X_train1.fillna(0).astype('float32'), y_train1,  #normalize data
                 validation_split=0.3, epochs=200, callbacks=[early_stopping_monitor])



#history=model.fit(X_train, y_train, validation_split=0.2, epochs=25)
score = mse_perceptron.evaluate(X_test1.fillna(0).astype('float32'), y_test1, verbose=0)
mse_perceptron.fit(X_train1.fillna(0).astype('float32'),y_train1.fillna(0))
fpr_list[3], tpr_list[3], _ = roc_curve(y_test1, y_roc(mse_perceptron,X_test1.fillna(0).astype('float32')))
print('MLP train Brier score:',
      brier_score(mse_perceptron.predict(X_train1.fillna(0).astype('float32')),y_train1),'\n test Brier score:',brier_score(mse_perceptron.predict(X_test1.fillna(0).astype('float32')),y_test1),
     '\n AUROC:',roc_auc_score(y_test1, y_roc(mse_perceptron,X_test1.fillna(0).astype('float32'))))

/Users/puddinpop/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
444/444 [==============================] - 3s 4ms/step - loss: 0.3160 - accuracy: 0.5057 - val_loss: 0.2606 - val_accuracy: 0.5278
Epoch 2/200
444/444 [==============================] - 1s 2ms/step - loss: 0.2912 - accuracy: 0.4941 - val_loss: 0.2544 - val_accuracy: 0.5190
Epoch 3/200
444/444 [==============================] - 1s 2ms/step - loss: 0.2724 - accuracy: 0.4965 - val_loss: 0.2517 - val_accuracy: 0.5389
Epoch 4/200
444/444 [==============================] - 1s 2ms/step - loss: 0.2600 - accuracy: 0.5002 - val_loss: 0.2506 - val_accuracy: 0.5288
Epoch 5/200
444/444 [==============================] - 1s 2ms/step - loss: 0.2539 - accuracy: 0.5021 - val_loss: 0.2502 - val_accuracy: 0.5117
Epoch 6/200
444/444 [==============================] - 1s 2ms/step - loss: 0.2514 - accuracy: 0.4958 - val_loss: 0.2501 - val_accuracy: 0.5278
Epoch 7/200
444/444 [==============================] - 1s 2ms/step - loss: 0.2505 - accuracy: 0.5021 - val_loss: 0.2500 - val_accuracy: 0.5182

Epoch 58/200
444/444 [==============================] - 1s 2ms/step - loss: 0.2500 - accuracy: 0.4907 - val_loss: 0.2500 - val_accuracy: 0.5022
Epoch 59/200
444/444 [==============================] - 1s 2ms/step - loss: 0.2500 - accuracy: 0.4931 - val_loss: 0.2500 - val_accuracy: 0.4942
Epoch 60/200
444/444 [==============================] - 1s 2ms/step - loss: 0.2500 - accuracy: 0.4941 - val_loss: 0.2500 - val_accuracy: 0.5062
Epoch 61/200
444/444 [==============================] - 1s 2ms/step - loss: 0.2500 - accuracy: 0.5021 - val_loss: 0.2500 - val_accuracy: 0.4933
Epoch 62/200
444/444 [==============================] - 1s 2ms/step - loss: 0.2500 - accuracy: 0.4883 - val_loss: 0.2500 - val_accuracy: 0.4822
Epoch 63/200
159/159 [==============================] - 0s 1ms/step
MLP train Brier score: 0.24999998861161443 
 test Brier score: 0.24999997526317796 
 AUROC: 0.5320192548992768


In [36]:
#Multi Layer Perceptron from Categorical Cross Entropy
    
cce_perceptron = Sequential()
n_cols = X_train1.shape[1]
cce_perceptron.add(BatchNormalization(input_shape=(n_cols,)))
cce_perceptron.add(Dense(70, activation= 'linear'))
cce_perceptron.add(Dropout(0.3))
cce_perceptron.add(Dense(50, activation= 'relu'))
cce_perceptron.add(Dropout(0.3))
cce_perceptron.add(Dense(50, activation= 'relu'))
cce_perceptron.add(Dropout(0.3))

#model.add(Dense(100, activation='relu'))
#model.add(Dropout(0.3))
#model.add(Dense(100, activation='linear'))
cce_perceptron.add(BatchNormalization())
cce_perceptron.add(Dense(2, activation='softmax'))
    #model.compile(
        #optimizer='Adam',
        
   
early_stopping_monitor = EarlyStopping(patience=50)
sgd = keras.optimizers.SGD(lr=.001, decay=2e-4, momentum=0.9, nesterov=True)
cce_perceptron.compile(loss = 'sparse_categorical_crossentropy',optimizer = 'sgd', metrics=['accuracy'])
history=cce_perceptron.fit(X_train1.fillna(0).astype('float32'), y_train1,  #normalize data
                 validation_split=0.3, epochs=200, callbacks=[early_stopping_monitor])



#history=model.fit(X_train, y_train, validation_split=0.2, epochs=25)
score = cce_perceptron.evaluate(X_test1.fillna(0).astype('float32'), y_test1, verbose=0)
cce_perceptron.fit(X_train1.fillna(0).astype('float32'),y_train1.fillna(0))
fpr_list[3], tpr_list[3], _ = roc_curve(y_test1, y_roc(cce_perceptron,X_test1.fillna(0).astype('float32')))
print('MLP train Brier score:',
      brier_score(cce_perceptron.predict(X_train1.fillna(0).astype('float32')),y_train1),'\n test Brier score:',brier_score(cce_perceptron.predict(X_test1.fillna(0).astype('float32')),y_test1),
     '\n AUROC:',roc_auc_score(y_test1, y_roc(cce_perceptron,X_test1.fillna(0).astype('float32'))))

Epoch 1/200


/Users/puddinpop/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


444/444 [==============================] - 2s 3ms/step - loss: 0.7092 - accuracy: 0.6377 - val_loss: 0.5643 - val_accuracy: 0.7135
Epoch 2/200
444/444 [==============================] - 1s 3ms/step - loss: 0.5949 - accuracy: 0.6950 - val_loss: 0.5589 - val_accuracy: 0.7133
Epoch 3/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5796 - accuracy: 0.7002 - val_loss: 0.5554 - val_accuracy: 0.7169
Epoch 4/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5714 - accuracy: 0.7060 - val_loss: 0.5520 - val_accuracy: 0.7186
Epoch 5/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5694 - accuracy: 0.7070 - val_loss: 0.5505 - val_accuracy: 0.7171
Epoch 6/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5695 - accuracy: 0.7082 - val_loss: 0.5484 - val_accuracy: 0.7210
Epoch 7/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5629 - accuracy: 0.7099 - val_loss: 0.5454 - val_accuracy: 0.7197
Epoch 8/200

444/444 [==============================] - 1s 3ms/step - loss: 0.5416 - accuracy: 0.7264 - val_loss: 0.5337 - val_accuracy: 0.7324
Epoch 59/200
444/444 [==============================] - 2s 4ms/step - loss: 0.5457 - accuracy: 0.7239 - val_loss: 0.5330 - val_accuracy: 0.7368
Epoch 60/200
444/444 [==============================] - 1s 3ms/step - loss: 0.5462 - accuracy: 0.7230 - val_loss: 0.5329 - val_accuracy: 0.7370
Epoch 61/200
444/444 [==============================] - 1s 3ms/step - loss: 0.5459 - accuracy: 0.7233 - val_loss: 0.5327 - val_accuracy: 0.7362
Epoch 62/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5440 - accuracy: 0.7259 - val_loss: 0.5322 - val_accuracy: 0.7353
Epoch 63/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5447 - accuracy: 0.7226 - val_loss: 0.5328 - val_accuracy: 0.7368
Epoch 64/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5463 - accuracy: 0.7223 - val_loss: 0.5324 - val_accuracy: 0.7381
Epoch

Epoch 115/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5424 - accuracy: 0.7250 - val_loss: 0.5314 - val_accuracy: 0.7362
Epoch 116/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5406 - accuracy: 0.7240 - val_loss: 0.5310 - val_accuracy: 0.7360
Epoch 117/200
444/444 [==============================] - 1s 3ms/step - loss: 0.5398 - accuracy: 0.7248 - val_loss: 0.5317 - val_accuracy: 0.7371
Epoch 118/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5433 - accuracy: 0.7212 - val_loss: 0.5316 - val_accuracy: 0.7401
Epoch 119/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5419 - accuracy: 0.7233 - val_loss: 0.5312 - val_accuracy: 0.7375
Epoch 120/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5385 - accuracy: 0.7288 - val_loss: 0.5322 - val_accuracy: 0.7350
Epoch 121/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5404 - accuracy: 0.7254 - val_loss: 0.5320 - val_ac

444/444 [==============================] - 1s 2ms/step - loss: 0.5388 - accuracy: 0.7273 - val_loss: 0.5300 - val_accuracy: 0.7409
Epoch 172/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5394 - accuracy: 0.7244 - val_loss: 0.5326 - val_accuracy: 0.7316
Epoch 173/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5409 - accuracy: 0.7254 - val_loss: 0.5307 - val_accuracy: 0.7376
Epoch 174/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5374 - accuracy: 0.7263 - val_loss: 0.5302 - val_accuracy: 0.7426
Epoch 175/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5382 - accuracy: 0.7270 - val_loss: 0.5305 - val_accuracy: 0.7368
Epoch 176/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5397 - accuracy: 0.7254 - val_loss: 0.5304 - val_accuracy: 0.7408
Epoch 177/200
444/444 [==============================] - 1s 2ms/step - loss: 0.5407 - accuracy: 0.7267 - val_loss: 0.5311 - val_accuracy: 0.7352

In [61]:
#Multi Layer Perceptron from Categorical Cross Entropy
    
cce_perceptron = Sequential()
n_cols = X_train1.shape[1]
cce_perceptron.add(BatchNormalization(input_shape=(n_cols,)))
cce_perceptron.add(Dense(1024, activation= 'relu'))
cce_perceptron.add(Dropout(0.5))
cce_perceptron.add(Dense(256, activation= 'relu'))
cce_perceptron.add(Dense(256, activation= 'relu'))
cce_perceptron.add(Dropout(0.5))
cce_perceptron.add(Dense(500, activation= 'relu'))
cce_perceptron.add(Dropout(0.3))

#model.add(Dense(100, activation='relu'))
#model.add(Dropout(0.3))
#model.add(Dense(100, activation='linear'))
cce_perceptron.add(BatchNormalization())
cce_perceptron.add(Dense(2, activation='softmax'))
    #model.compile(
        #optimizer='Adam',
        
   
early_stopping_monitor = EarlyStopping(patience=50)
sgd = keras.optimizers.SGD(lr=.001, decay=2e-4, momentum=0.9, nesterov=True)
cce_perceptron.compile(loss = 'sparse_categorical_crossentropy',optimizer = 'sgd', metrics=['accuracy'])
history=cce_perceptron.fit(X_train1.fillna(0).astype('float32'), y_train1,  #normalize data
                 validation_split=0.3, epochs=50, callbacks=[early_stopping_monitor])



#history=model.fit(X_train, y_train, validation_split=0.2, epochs=25)
score = cce_perceptron.evaluate(X_test1.fillna(0).astype('float32'), y_test1, verbose=0)
cce_perceptron.fit(X_train1.fillna(0).astype('float32'),y_train1.fillna(0))
fpr_list[3], tpr_list[3], _ = roc_curve(y_test1, y_roc(cce_perceptron,X_test1.fillna(0).astype('float32')))
print('MLP train Brier score:',
      brier_score(cce_perceptron.predict(X_train1.fillna(0).astype('float32')),y_train1),'\n test Brier score:',brier_score(cce_perceptron.predict(X_test1.fillna(0).astype('float32')),y_test1),
     '\n AUROC:',roc_auc_score(y_test1, y_roc(cce_perceptron,X_test1.fillna(0).astype('float32'))))

Epoch 1/50


/Users/puddinpop/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


444/444 [==============================] - 4s 7ms/step - loss: 0.6938 - accuracy: 0.6364 - val_loss: 0.5515 - val_accuracy: 0.7276
Epoch 2/50
444/444 [==============================] - 2s 6ms/step - loss: 0.5920 - accuracy: 0.6881 - val_loss: 0.5584 - val_accuracy: 0.7110
Epoch 3/50
444/444 [==============================] - 3s 6ms/step - loss: 0.5714 - accuracy: 0.7040 - val_loss: 0.5354 - val_accuracy: 0.7373
Epoch 4/50
444/444 [==============================] - 3s 6ms/step - loss: 0.5640 - accuracy: 0.7145 - val_loss: 0.5361 - val_accuracy: 0.7340
Epoch 5/50
444/444 [==============================] - 3s 6ms/step - loss: 0.5563 - accuracy: 0.7171 - val_loss: 0.5338 - val_accuracy: 0.7367
Epoch 6/50
444/444 [==============================] - 3s 8ms/step - loss: 0.5573 - accuracy: 0.7128 - val_loss: 0.5336 - val_accuracy: 0.7376
Epoch 7/50
444/444 [==============================] - 3s 6ms/step - loss: 0.5562 - accuracy: 0.7161 - val_loss: 0.5345 - val_accuracy: 0.7409
Epoch 8/50
444/44

In [4]:
#Multi Layer Perceptron from Categorical Cross Entropy and Optimizer Adam
    
cce_adam = Sequential()

n_cols = X_train1.shape[1]

cce_adam.add(BatchNormalization(input_shape=(n_cols,)))
cce_adam.add(Dense(70, activation= 'linear'))
cce_adam.add(Dropout(0.3))
cce_adam.add(Dense(50, activation= 'relu'))
cce_adam.add(Dropout(0.3))
cce_adam.add(Dense(50, activation= 'relu'))
cce_adam.add(Dropout(0.3))

#model.add(Dense(100, activation='relu'))
#model.add(Dropout(0.3))
#model.add(Dense(100, activation='linear'))
cce_adam.add(BatchNormalization())
cce_adam.add(Dense(2, activation='softmax'))
    
        
   
early_stopping_monitor = EarlyStopping(patience=50)
adam = keras.optimizers.Adam(lr=.001, decay=2e-4)
cce_adam.compile(loss = 'sparse_categorical_crossentropy',optimizer = 'adam', metrics=['accuracy'])
history=cce_adam.fit(X_train1.fillna(0).astype('float32'), y_train1,  #normalize data
                 validation_split=0.3, epochs=200, callbacks=[early_stopping_monitor])



#history=model.fit(X_train, y_train, validation_split=0.2, epochs=25)
score = cce_adam.evaluate(X_test1.fillna(0).astype('float32'), y_test1, verbose=0)
cce_adam.fit(X_train1.fillna(0).astype('float32'),y_train1.fillna(0))
fpr_list[3], tpr_list[3], _ = roc_curve(y_test1, y_roc(cce_adam,X_test1.fillna(0).astype('float32')))
print('MLP train Brier score:',
      brier_score(cce_adam.predict(X_train1.fillna(0).astype('float32')),y_train1),'\n test Brier score:',brier_score(cce_adam.predict(X_test1.fillna(0).astype('float32')),y_test1),
     '\n AUROC:',roc_auc_score(y_test1, y_roc(cce_adam,X_test1.fillna(0).astype('float32'))))

NameError: name 'Sequential' is not defined

### Testing

In [2]:
testdata = recidivism(X_test_np, y_test_np)
outputs = clf(testdata.X)
__, predicted = torch.max(outputs, 1)

NameError: name 'recidivism' is not defined

In [ ]:
plt.plot(loss_arr)
plt.xlabel("epoch")
plt.ylabel("running loss")
plt.title(f"SGD, batch size = {batch_size}")